In [1]:
# data_parsing.py

# --- Imports ---
import pandas as pd
import os
import re

# --- Set Working Directory ---
os.chdir(r"C:\Users\manoh\Downloads\Tech Challange")

# --- Load CSV File ---
file_path = "Data for Technical Challenge.csv"
df = pd.read_csv(file_path)

# --- Step 1: Convert Date Columns to datetime ---
df['Date Invoiced'] = pd.to_datetime(df['Date Invoiced'], errors='coerce')
df['Date Paid'] = pd.to_datetime(df['Date Paid'], errors='coerce')

# --- Step 2: Clean Invoice Reference ---

def extract_clean_ref_from_string(ref_raw, date_invoiced):
    """
    Extract a clean and consistent invoice reference from messy input strings.
    Based on the invoice year and following numbers.
    """
    invoice_year = date_invoiced.year if pd.notnull(date_invoiced) else 'XXXX'
    yy = str(invoice_year)[-2:]
    digit_chunks = re.findall(r'\d+', str(ref_raw))
    ref_number = None

    for idx, chunk in enumerate(digit_chunks):
        if chunk == str(invoice_year) or chunk == yy:
            if idx + 1 < len(digit_chunks):
                ref_number = digit_chunks[idx + 1]
                break

    if not ref_number:
        if len(digit_chunks) == 1 and digit_chunks[0].startswith(yy):
            ref_number = digit_chunks[0][len(yy):] or '0000'
        else:
            ref_number = digit_chunks[-1] if digit_chunks else '0000'

    return f"{invoice_year}-{ref_number}"

# --- Step 3: Apply Cleaning to Invoice References ---
df['Clean Invoice Ref'] = df.apply(lambda row: extract_clean_ref_from_string(row['Invoice Reference'], row['Date Invoiced']), axis=1)
df.drop(columns=['Invoice Reference'], inplace=True)

# --- Step 4: Format Dates to String Format ---
df['Date Invoiced'] = df['Date Invoiced'].dt.strftime('%Y-%m-%d')
df['Date Paid'] = df['Date Paid'].dt.strftime('%Y-%m-%d')

# --- Step 5: Extract and Map Client IDs ---
df['Client ID'] = df['Client Name']
unique_clients = sorted(df['Client ID'].dropna().unique(), key=lambda x: int(x) if str(x).isdigit() else float('inf'))
client_mapping = {client: f"Client {i+1}" for i, client in enumerate(unique_clients)}
df['Client Name'] = df['Client ID'].map(client_mapping)

# --- Step 6: Fill Missing Invoice and Payment Amounts ---
df['Invoice Amount'] = df['Invoice Amount'].fillna(0)
df['Paid Amount'] = df['Paid Amount'].fillna(0)

# --- Step 7: Sort Data for Proper Cumulative Calculations ---
df = df.sort_values(by=['Client ID', 'Clean Invoice Ref', 'Date Invoiced', 'Date Paid'])

# --- Step 8: Calculate Cumulative Paid and Pending Amounts ---
df['Cumulative Paid'] = 0.0
df['Pending Amount'] = 0.0
group_cols = ['Client ID', 'Clean Invoice Ref', 'Invoice Amount', 'Date Invoiced']
df['_group_id'] = df.groupby(group_cols).ngroup()
cumulative_paid_map = {}

for idx, row in df.iterrows():
    group_key = (row['Client ID'], row['Clean Invoice Ref'], row['Invoice Amount'], row['Date Invoiced'])
    cumulative = cumulative_paid_map.get(group_key, 0.0)
    current_paid = row['Paid Amount']
    new_cumulative = cumulative + current_paid

    df.at[idx, 'Cumulative Paid'] = new_cumulative
    df.at[idx, 'Pending Amount'] = round(float(row['Invoice Amount']) - new_cumulative, 2)

    cumulative_paid_map[group_key] = new_cumulative

# --- Step 9: Identify Possible Duplicate Payments ---
dup_keys = ['Client ID', 'Clean Invoice Ref', 'Date Invoiced', 'Invoice Amount', 'Paid Amount', 'Date Paid']
df['Is Possible Duplicate'] = df.duplicated(subset=dup_keys, keep='first')

# --- Step 10: Mark Late Payments ---
df['Is Late'] = (pd.to_datetime(df['Date Paid'], errors='coerce') - pd.to_datetime(df['Date Invoiced'], errors='coerce')).dt.days > 30

# --- Step 11: Rename Days to Pay Column (if it exists) ---
if 'No. Days taken to Pay' in df.columns:
    df.rename(columns={'No. Days taken to Pay': 'Days to Pay'}, inplace=True)

# --- Step 12: Drop Internal Group ID ---
df.drop(columns=['_group_id'], inplace=True)

# --- Step 13: Generate Clean and Unique Invoice Group IDs ---
invoice_group_ids = (
    df.groupby(['Client Name', 'Clean Invoice Ref', 'Invoice Amount', 'Date Invoiced'])
      .ngroup()
)
df['Invoice Group Index'] = invoice_group_ids

# Create a readable Invoice Group ID like Client1_1, Client1_2, etc.
df['Invoice Group ID'] = df.groupby('Client Name')['Invoice Group Index'].transform(lambda x: pd.factorize(x)[0] + 1)
df['Invoice Group ID'] = df.apply(lambda row: f"{row['Client Name'].replace(' ', '')}_{row['Invoice Group ID']}", axis=1)

# --- Step 14: Reorder Columns for Output Consistency ---
ordered_columns = ['Invoice Group ID', 'Client ID', 'Client Name'] + [col for col in df.columns if col not in ['Client ID', 'Client Name', 'Invoice Group ID']]
df = df[ordered_columns]

# --- Step 15: Save Cleaned Data to JSON ---
df.to_json("cleaned_data.json", orient="records", indent=2)

# Step 15b: Save as CSV
#df.to_csv("cleaned_data.csv", index=False, encoding='utf-8-sig')

# --- Step 16: Preview DataFrame Head (optional) ---
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
display(df.head())


,Invoice Group ID,Client ID,Client Name,Date Invoiced,Date Paid,Days to Pay,Invoice Amount,Paid Amount,Clean Invoice Ref,Cumulative Paid,Pending Amount,Is Possible Duplicate,Is Late,Invoice Group Index
27,Client1_1,1,Client 1,2018-03-29,2018-07-18,111,539101.90,539101.90,2018-4222,539101.90,0.00,False,True,1
28,Client1_2,1,Client 1,2018-03-29,2018-07-18,111,41447.58,41447.58,2018-4222,41447.58,0.00,False,True,0
82,Client1_3,1,Client 1,2018-05-31,2018-07-18,48,637127.30,637127.30,2018-4304,637127.30,0.00,False,True,3
83,Client1_4,1,Client 1,2018-05-31,2018-07-18,48,90633.41,90633.41,2018-4304,90633.41,0.00,False,True,2
12,Client1_5,1,Client 1,2018-06-28,2019-01-09,195,308482.12,308482.12,2018-4330,308482.12,0.00,False,True,5


In [ ]:
#http://127.0.0.1:5000/api/clients
#http://127.0.0.1:5000/api/invoices
#http://127.0.0.1:5000/api/invoices/Client%207